
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.2 - Trilobite preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🐬🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🐬🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊

<p>


This script is used to check your local docker images.

Two methods can be used - shell commands with subprocess. Popen, or
use the docker SDK (docker-py) to manage and inspect local configuration and running images.

A function is declared to wrap executing a command insider an image which retrieves the contract addresses
from the contract artifact JSON files (ABIs). Use this to confirm that your local configuration (as
specified in config_local.ini) matches the deployed images.

In [ ]:
import docker
import subprocess
# import subprocess
# import pathlib
# import squid_py.ocean as ocean
import sys
from pprint import pprint

In [ ]:
import logging

loggers_dict = logging.Logger.manager.loggerDict

logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.DEBUG)

# FORMAT = "%(asctime)s - %(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"
# FORMAT = "%(asctime)s %(levelno)s: %(module)30s %(message)s"
FORMAT = "%(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"

DATE_FMT = "%Y-%m-%d %H:%M:%S"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logger.debug("Logging started")


class LoggerCritical:
    def __enter__(self):
        my_logger = logging.getLogger()
        my_logger.setLevel("CRITICAL")

    def __exit__(self, type, value, traceback):
        my_logger = logging.getLogger()
        my_logger.setLevel("DEBUG")

In [ ]:
# s = subprocess.check_output('docker ps', shell=True).wait()
s = subprocess.Popen("docker ps" + "", shell=True).wait()
print(s)

In [ ]:
# High level client
client = docker.from_env()
# Get the APIClient for running commands
low_level_api_client = docker.APIClient(base_url='unix://var/run/docker.sock')

In [ ]:
with LoggerCritical():
    for container in client.containers.list():
        print(f"Docker container {container.name} is {container.status}")
        print('\tTags:', container.image.tags)
        # print(container.labels)
        # print("\n")
    # container.logs()

In [ ]:
def get_address(api_client, container_id,contract_name):
    network_name = 'ocean_poa_net_local'
    # This is the python script to be executed in the running image
    python_script = r"import sys, json; print(json.load(open('/keeper-contracts/artifacts/{}.{}.json', 'r'))['address'])".format(contract_name,network_name)

    # Wrap the script in quotes (string) and add the python shell command
    command = r"python -c " + '"' + python_script + '"'

    # Create and run the command
    ex = api_client.exec_create(container=container_id, cmd=command)

    return api_client.exec_start(ex)

# Get the docker image running the smart contracts, by searching on the name
container_keeper_contracts = [c for c in client.containers.list() if 'keeper-contracts' in c.name][0]

addresses=dict()
addresses['market.address'] = get_address(low_level_api_client, container_keeper_contracts.id,'OceanMarket').decode("utf-8").rstrip()
addresses['auth.address'] = get_address(low_level_api_client, container_keeper_contracts.id,'OceanAuth').decode("utf-8").rstrip()
addresses['token.address'] = get_address(low_level_api_client, container_keeper_contracts.id,'OceanToken').decode("utf-8").rstrip()
addresses['didregistry.address'] = get_address(low_level_api_client, container_keeper_contracts.id,'DIDRegistry').decode("utf-8").rstrip()

print("Artifact addresses retrieved:")
pprint(addresses)